In [1]:
import sqlite3

In [2]:
#Create a db
#Return the database as a Connection object
#Takes only the name as an argument
conn = sqlite3.connect('hello.db')

In [ ]:
conn = sqlite3.connect(':memory:') #Create a database in RAM

In [3]:
#Cursor objects allow any SQL statement to be executed in the database
c = conn.cursor()

#Create a table in the database named hello
c.execute('''CREATE TABLE IF NOT EXISTS tasks (
id INTEGER PRIMARY KEY,
name TEXT NOT NULL,
priority INTEGER NOT NULL
);''')

In [4]:
#Insert data into the table

#Id does not need to be mentioned, unique primary keys will be self-generated/are auto-incremental
#c.execute('''INSERT INTO tasks (id, name, priority) VALUES (1, 'My first task', 1);''')
#(?,?) format to avoid SQL injection attack?
c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', ('My first task', 1))
#Commit the change made
conn.commit()

In [5]:
#Use the executemany command to repeat command multiple times
#Open the db
c = conn.cursor()

tasks = [
    ('My first task', 1),
    ('My second task', 5),
    ('My third task', 10),
]
c.executemany('INSERT INTO tasks (name, priority) VALUES (?,?)', tasks)
conn.commit()

In [86]:
#Refactored approach
def start_menu():
    print('1. Show Tasks \n2. Add Task \n3. Change Priority \n4. Delete Task \n5. Exit')
    user_choice = input('Which action do you want to perform?')
    return user_choice

class Todo:
    def __init__(self):
        self.conn = sqlite3.connect('todo.db')
        self.c = self.conn.cursor()
        self.create_task_table()
        
    def create_task_table(self):
        self.c.execute('''CREATE TABLE IF NOT EXISTS tasks (
                     id INTEGER PRIMARY KEY,
                     name TEXT NOT NULL,
                     priority INTEGER NOT NULL
                     );''')
    
    def find_task(self, name_find):
        c = self.c.execute('SELECT id, name, priority FROM tasks WHERE name == (?)', (name_find,))
        rows = c.fetchall()
        if rows == []:
            return None
        else:
            return rows[0]
        
    def add_task(self):
        name = input('Enter task name: ')
        if name == '':
            print('Nothing was entered')
            return
        
        task_with_same_name = self.find_task(name)
        if task_with_same_name != None:
            print('There already exists a task with this name')
            return
        
        priority = int(input('Enter priority: '))
        if priority < 1:
            print('Only priorities allowed values 1 and up')
            return
        
        self.c.execute('INSERT INTO tasks (name, priority) VALUES (?,?)', (name, priority))
        self.conn.commit()
    
    def show_tasks(self):
        c = self.c.execute('SELECT id, name, priority FROM tasks')
        rows = c.fetchall()
        for row in rows:
            print(row)
        return 
    
    def change_priority(self):
        task_id = input('For which task do you want to change the priority? Enter id:')
        new_priority = input('What do you want to change the priority to?')
        if int(new_priority) < 1:
            print('Only priorities allowed values 1 and up')
            return
        
        self.c.execute('UPDATE tasks SET priority = ? WHERE id = ?', (new_priority, task_id))
        self.conn.commit()
        
    def delete_task(self):
        task_id = input('Enter task id to delete:')
        
        self.c.execute('DELETE FROM tasks WHERE id = ?', (task_id,))
        self.conn.commit()

In [93]:
app = Todo()
user_choice = start_menu()

if user_choice == '1':
    app.show_tasks()
elif user_choice == '2':
    app.add_task()
elif user_choice == '3':
    app.change_priority()
elif user_choice == '4':
    app.delete_task()
elif user_choice == '5':
    print('exit')
else:
    print('This answer was not part of the menu')

1. Show Tasks 
2. Add Task 
3. Change Priority 
4. Delete Task 
5. Exit


Which action do you want to perform? 5


exit


In [80]:
app.delete_task()

Enter task id to delete: 4


In [77]:
app.change_priority()

For which task do you want to change the priority? Enter id: 2
What do you want to change the priority to? 100


In [71]:
c = app.c
c.execute('UPDATE tasks SET priority = ?', (20, ))
app.conn.commit()

In [73]:
c.execute('DELETE FROM tasks WHERE id = ?', (1,)) #Ensure a tuple when only inserting one variable
app.conn.commit()

In [69]:
app.add_task()

Enter task name:  hi
Enter priority:  5


In [81]:
app.show_tasks()

(2, 'do washing', 100)
(3, 'run outside', 20)
(5, 'helpie', 20)
(6, '1', 20)
(7, 'help', 20)
(8, 'lala', 20)
(9, 'x', 20)
(10, 'ff', 20)
(11, 'hi', 20)


In [4]:
app.add_task()

Enter task name:  helpie
Enter priority:  5


In [5]:
#Reading data using the SELECT command
for row in app.c.execute('''SELECT * FROM tasks;'''):
    print(row)

(1, 'task55', 4)
(2, 'do washing', 1)
(3, 'run outside', 2)
(4, 'test', 2)
(5, 'helpie', 5)


In [6]:
c = app.c.execute('SELECT * FROM tasks')
rows = c.fetchall() #Collect all records fetched to avoid using the execute command as an iterator
for row in rows:
    print(row)

(1, 'task55', 4)
(2, 'do washing', 1)
(3, 'run outside', 2)
(4, 'test', 2)
(5, 'helpie', 5)


In [7]:
c = app.c.execute('SELECT * FROM tasks')
for row in rows:
    rows = c.fetchone() #Collect the next record
    print(row)

(1, 'task55', 4)
(2, 'do washing', 1)
(3, 'run outside', 2)
(4, 'test', 2)
(5, 'helpie', 5)


In [26]:
id_find = 2

c = app.c.execute('SELECT id, name FROM tasks WHERE id == ?', str(id_find))
rows = c.fetchall()
names = [x[0] for x in rows]
rows
# if find_name in names:
#     return 

[(2, 'do washing')]

In [46]:
name_find = 'test'

def find_task(name_find):
    c = app.c.execute('SELECT id, name, priority FROM tasks WHERE name == (?)', (name_find,))
    rows = c.fetchall()
    if rows == []:
        return None
    else:
        return rows[0]
    
find_task(name_find)

(4, 'test', 2)